## Parquet to Kafka streamlike, continuously

In [1]:
import csv
import re
import math
import time
import random
import numpy as np
import sys
import json
import requests
import os
from datetime import timedelta

In [2]:
import threading, logging, time
import multiprocessing

from kafka import KafkaConsumer, KafkaProducer

kafkaUrl = "smartaqnet-dev01.teco.edu"




In [3]:
data = {
  "phenomenonTime": "2016-03-01T22:50:00.000Z",
  "resultTime" : "2016-03-01T22:50:00.000Z",
  "result" : "1.8",
  "FeatureOfInterest": {
      "@iot.id": "t1edu.teco.wang/IBADENWR82/FoI/IBADENWR82/48.441/9.257"
  }
}
json.dumps(data)

'{"phenomenonTime": "2016-03-01T22:50:00.000Z", "FeatureOfInterest": {"@iot.id": "t1edu.teco.wang/IBADENWR82/FoI/IBADENWR82/48.441/9.257"}, "resultTime": "2016-03-01T22:50:00.000Z", "result": "1.8"}'

In [4]:
# dirList: sorted list of dir names
dirList = os.listdir("./out/outAllSortByTimeStampAndIDBigAll/")
dirList = [ x for x in dirList if ("2016" in x) or ("2015" in x) ]
dirList = sorted(dirList, key= lambda myDir: int(myDir.split("=")[1]))



In [5]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
#spark conf
conf = ( SparkConf()
         .setMaster("local[*]")
         .setAppName('pyspark')
        )
ss = SparkSession.builder.config(conf=conf).getOrCreate()
sc = ss.sparkContext


In [6]:
#Common things
mytypes = ["Air-Temperature","Air-Humidity","Air-Pressure"]
urlHome = 'http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0'
#urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
urlSensors = urlHome + '/Sensors'
urlObservedProperty = urlHome + '/ObservedProperties'
urlDataStream = urlHome + '/Datastreams'
urlFoI = urlHome + '/FeaturesOfInterest'

In [7]:
urlInst = "edu.teco.wang"
urlInstBroadCated = sc.broadcast(urlInst)
#time.sleep(30)

In [8]:
def parseTime(aRow):
    from datetime import time
    aTime = time(int(aRow.Time[0:2]),int(aRow.Time[3:5]),int(aRow.Time[6:]))
    #return (aTime,aRow)
    return (int(aRow.Time[0:2]),aRow)
def genDatastreamUID(aValue):
    key = aValue[0]
    aValue = aValue[1]
    key = urlInstBroadCated.value + "-"+aValue.ID + "-Datastreams-" + key + "-" + aValue.ID
    aValue = aValue.asDict()
    return (key,aValue)
def fireToKafka(aRow):
    from kafka import KafkaConsumer, KafkaProducer
    producer = KafkaProducer(bootstrap_servers=kafkaUrl+':9092')
    #str(aRow[1])
    future = producer.send("WG_Measurements",key = str(aRow[0]),value = str(aRow[1]))
    future.get(timeout=10)
    return aRow[0]

In [10]:
kafkaUrl

'smartaqnet-dev01.teco.edu'

In [11]:
#Each myDir represents a date
producer = KafkaProducer(bootstrap_servers=kafkaUrl+':9092')
for myDir in dirList:
    #print(myDir)
    #myDir = dirList[0]
    inputDir = "./out/outAllSortByTimeStampAndIDBigAll/"+myDir
    dataFileDF = ss.read.option("basepath",inputDir).parquet(inputDir)#+"TimeStamp=20160504/ID=I72406BI1")
    dataFileDF = dataFileDF.withColumnRenamed("AitTemperature","AirTemperature")
    dataFileDF = dataFileDF.filter("(not isnull(ID)) and ID !=''")
    idDF = dataFileDF["ID","Latitude","Longitude"]
    indexedDataFileDF = dataFileDF.select("ID","Time","Latitude","Longitude","AirTemperature")#.map(lambda x: (x.Time,{x.ID,x.Latitude,x.Longitude,x.AirPressure}))
    indexedDataFileDF = indexedDataFileDF.rdd.map(parseTime)#.groupByKey().mapValues(list)
    for hourInt in range(0,24):
        #hourInt = 0
        print(myDir + " " + str(hourInt))
        #future = producer.send("MyGlobalTime",value = myDir + " " + str(hourInt))
        #future.get(timeout=10)
        fireToKafkaRDD = indexedDataFileDF.filter(lambda v:v[0]==hourInt)#.flatMapValues(lambda v:v)
        #indexedDataFileDF = indexedDataFileDF.map(parseTime)
        fireToKafkaRDD = fireToKafkaRDD.map(lambda v:genDatastreamUID(("AirTemperature",v[1])))
        fireToKafkaRDD = fireToKafkaRDD.map(fireToKafka)
        fireToKafkaRDD.collect()
    time.sleep(10)

TimeStamp=20150101 0
TimeStamp=20150101 1
TimeStamp=20150101 2
TimeStamp=20150101 3
TimeStamp=20150101 4
TimeStamp=20150101 5
TimeStamp=20150101 6
TimeStamp=20150101 7
TimeStamp=20150101 8
TimeStamp=20150101 9
TimeStamp=20150101 10
TimeStamp=20150101 11
TimeStamp=20150101 12
TimeStamp=20150101 13
TimeStamp=20150101 14
TimeStamp=20150101 15
TimeStamp=20150101 16
TimeStamp=20150101 17
TimeStamp=20150101 18


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 49 in stage 20.0 failed 1 times, most recent failure: Lost task 49.0 in stage 20.0 (TID 4677, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/wang/anaconda3/envs/kafkTest27/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/home/wang/anaconda3/envs/kafkTest27/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/wang/anaconda3/envs/kafkTest27/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-8-23fc49081596>", line 14, 

In [11]:
producer = KafkaProducer(bootstrap_servers=kafkaUrl+':9092')

#print(myDir)
myDir = dirList[0]
inputDir = "./out/outAllSortByTimeStampAndIDBigAll/"+myDir
dataFileDF = ss.read.option("basepath",inputDir).parquet(inputDir)#+"TimeStamp=20160504/ID=I72406BI1")
dataFileDF = dataFileDF.withColumnRenamed("AitTemperature","AirTemperature")
dataFileDF = dataFileDF.filter("(not isnull(ID)) and ID !=''")
idDF = dataFileDF["ID","Latitude","Longitude"]
indexedDataFileDF = dataFileDF.select("ID","Time","Latitude","Longitude","AirTemperature")#.map(lambda x: (x.Time,{x.ID,x.Latitude,x.Longitude,x.AirPressure}))
indexedDataFileDF = indexedDataFileDF.rdd.map(parseTime)#.groupByKey().mapValues(list)


#future = producer.send("MyGlobalTime",value = myDir + " " + str(hourInt))
#future.get(timeout=10)
fireToKafkaRDD = indexedDataFileDF.filter(lambda v:v[0]==v[0])#.flatMapValues(lambda v:v)
#indexedDataFileDF = indexedDataFileDF.map(parseTime)
fireToKafkaRDD = fireToKafkaRDD.map(lambda v:genDatastreamUID(("AirTemperature",v[1]))[0])
#fireToKafkaRDD = fireToKafkaRDD.map(fireToKafka)

In [33]:
bigList = fireToKafkaRDD.map(lambda x:(x,x)).reduceByKey(lambda x,y:1).map(lambda x:x[0]).reduce(lambda x,y:x+"," +y)
bigList

u'edu.teco.wang-IESSEN68-Datastreams-AirTemperature-IESSEN68,edu.teco.wang-IWOLPERT2-Datastreams-AirTemperature-IWOLPERT2,edu.teco.wang-IMELSUNG7-Datastreams-AirTemperature-IMELSUNG7,edu.teco.wang-INIEDERS122-Datastreams-AirTemperature-INIEDERS122,edu.teco.wang-ILICHTEN5-Datastreams-AirTemperature-ILICHTEN5,edu.teco.wang-IBAYERNN3-Datastreams-AirTemperature-IBAYERNN3,edu.teco.wang-IGRAFENA2-Datastreams-AirTemperature-IGRAFENA2,edu.teco.wang-IBLUMBER2-Datastreams-AirTemperature-IBLUMBER2,edu.teco.wang-INRWPULH1-Datastreams-AirTemperature-INRWPULH1,edu.teco.wang-IBERLIN322-Datastreams-AirTemperature-IBERLIN322,edu.teco.wang-INORDRHE100-Datastreams-AirTemperature-INORDRHE100,edu.teco.wang-IDIETZEN5-Datastreams-AirTemperature-IDIETZEN5,edu.teco.wang-IMECKLEN7-Datastreams-AirTemperature-IMECKLEN7,edu.teco.wang-ISCHWEDT3-Datastreams-AirTemperature-ISCHWEDT3,edu.teco.wang-ISEMBACH2-Datastreams-AirTemperature-ISEMBACH2,edu.teco.wang-ISPEYER9-Datastreams-AirTemperature-ISPEYER9,edu.teco.wang-IO

In [28]:
myRdd = sc.parallelize(range(1,5))
myRdd.map(lambda x:(1,str(x))).reduceByKey(lambda x,y:x+","+y).collect()

[(1, '1,2,3,4')]

In [22]:
fireToKafkaRDD = fireToKafkaRDD.map(fireToKafka)
fireToKafkaRDD.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 39, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/wang/anaconda3/envs/kafkTest27/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/home/wang/anaconda3/envs/kafkTest27/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/wang/anaconda3/envs/kafkTest27/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-21-de841f6bcbe7>", line 15, in 

In [ ]:
indexedDataFileDF.collect()

In [88]:
indexedDataFileDF.collect()[0][1]
range(0,23)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22]

In [81]:
astring = "2015-01-28"
bstring = "12:23:34"

bTime = datetime(int(astring[0:4]),int(astring[5:7]),int(astring[8:]),int(bstring[0:2]),int(bstring[3:5]),int(bstring[6:]))
bTime

datetime.datetime(2015, 1, 28, 12, 23, 34)

(2015, 1, 28, 12, 23, 34)

In [82]:
from datetime import timedelta
newBTime = timedelta(hours = 9) + bTime
newBTime

datetime.datetime(2015, 1, 28, 21, 23, 34)

In [72]:
bTime -bTime

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [8]:
from pyspark.sql import functions as F

from datetime import tzinfo, timedelta, datetime
for myDir in dirList:
    inputDir = "./out/outAllSortByTimeStampAndIDBig/"+myDir
    dataFileDF = ss.read.option("basepath",inputDir).parquet(inputDir)#+"TimeStamp=20160504/ID=I72406BI1")
    dataFileDF = dataFileDF.withColumnRenamed("AitTemperature","AirTemperature")
    dataFileDF = dataFileDF.filter("(not isnull(ID)) and ID !=''")
    idDF = dataFileDF["ID","Latitude","Longitude"]
    
    urlInst = "edu.teco.wang"
    idDF = idDF.withColumn("LatLon",F.array("Latitude","Longitude"))
    idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))

    idDF = idDF.withColumn("ThUID",F.format_string("%s/%s",F.lit(urlInst),idDF.ID))
    idDF = idDF.withColumn("DsUID",F.format_string("%s/Datastreams/%s",F.lit(urlInst),idDF.ID))
    
    for row in idDF.collect():
        #Thing

        data =  {
            "name": "DWD-Sensor-" + row.ID,
            "description": "This is DWD-Sensor-" + row.ID,
            "@iot.id": row.ThUID,
            "Locations": [
            {
                "name": row.ThUID,
                "description": "This is the location of DWD-Sensor-" + row.ID,
                "encodingType": "application/vnd.geo+json",
                "location": {
                  "type": "Point",
                  "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
                },
                "@iot.id": "Location/" + row.ThUID 
            }
          ]
            }
        p = requests.post(urlThings, json.dumps(data))
        if (p.status_code  == 201):
            print(201)
        else:
            print("Error:", p.status_code)

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)

        #Sensor

        deviceAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row.ID
        for mytype in mytypes:
            data = {
                "name": "DWD-Sensor-" + row.ID + "-" + "" + mytype,
                "description": "This is a Sensor from Netatmo Weather Station",
                "encodingType": "application/pdf",
                "metadata": deviceAddr,
                "@iot.id": row.ThUID +"-"+ mytype
            }
            p = requests.post(urlSensors, json.dumps(data))
            if (p.status_code  == 201):
                print(201)
            else:
                print("Error:", p.status_code)

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)

        #DataStream

        for mytype in mytypes:
            data = {
              "name": mytype + "-DS",
              "description": "Datastream for recording temperature",
              "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
                "@iot.id":row.ThUID + "/Datastreams/" + mytype + "-" + row.ID,
              "unitOfMeasurement": {
                "name": "Degree Celsius",
                "symbol": "degC",
                "definition": "http://www.qudt.org/qudt/owl/1.0.0/unit/Instances.html#DegreeCelsius"
              },
              "Thing":{"@iot.id":row.ThUID},
              "ObservedProperty":{"@iot.id":"/ObservedProperty/" + mytype},
              "Sensor":{"@iot.id":row.ThUID +"-"+ mytype}
            }
            p = requests.post(urlDataStream, json.dumps(data))
            if (p.status_code  == 201):
                print(201)
            else:
                print("Error:", p.status_code)

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)

        # FoI
        data = {
          "name": "Weather Station-" + row.ID,
          "description": "Weather Station-" + row.ID,
          "encodingType": "application/vnd.geo+json",
            "@iot.id": row.ThUID + "/FoI/"+row.ID,
          "feature": {
            "type": "Point",
            "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
          }
        }
        p = requests.post(urlFoI, json.dumps(data))
        if (p.status_code  == 201):
            print(201)
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)
    for row in dataFileDF.collect():
        time.sleep(0.5)

        class TZ(tzinfo):
            def utcoffset(self, dt): return timedelta(minutes=120)
        isoTime =datetime(int(row.Date.split('-')[0]),
                     int(row.Date.split("-")[1]),
                     int(row.Date.split("-")[2]),
                     int(row.Time.split(':')[0]),
                     int(row.Time.split(':')[1]),
                     int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()




        urlObs = urlHome+"/Datastreams('edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"
        data ={
          "phenomenonTime": isoTime,
          "resultTime" : isoTime,
          "result" : row.AirTemperature
        }
        p = requests.post(urlObs, json.dumps(data))
        if (p.status_code  == 201):
            print(p.headers["location"])
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)
    time.sleep(1)
    

('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
http://localhost:8080/FROST-Server/v1.0/Observations('4cf9931c-5525-11e8-92e3-a797254bd30a')
http://localhost:8080/FROST-Server/v1.0/Observations('4d4a9bcc-5525-11e8-92e3-1f293f896544')
http://localhost:8080/FROST-Server/v1.0/Observations('4d9b0b3e-5525-11e8-92e3-3f7cc6cba407')
http://localhost:8080/FROST-Server/v1.0/Observations('4deee088-5525-11e8-92e3-0fb6010b5d5e')
http://localhost:8080/FROST-Server/v1.0/Observations('4e40e4a0-5525-11e8-92e3-63301403ca26')
http://localhost:8080/FROST-Server/v1.0/Observations('4e9f0e72-5525-11e8-92e3-cb1dc55220dc')
http://localhost:8080/FROST-Server/v1.0/Observations('4eeffed6-5525-11e8-92e3-932fc9caecf6')
http://localhost:8080/FROST-Server/v1.0/Obser

http://localhost:8080/FROST-Server/v1.0/Observations('68cb431a-5525-11e8-92e3-77d63b027e99')
http://localhost:8080/FROST-Server/v1.0/Observations('691ef9a6-5525-11e8-92e3-1b6a5583911c')
http://localhost:8080/FROST-Server/v1.0/Observations('69745270-5525-11e8-92e3-ab673d3a4750')
http://localhost:8080/FROST-Server/v1.0/Observations('69c459b4-5525-11e8-92e3-c75bcd65beed')
http://localhost:8080/FROST-Server/v1.0/Observations('6a1c08f8-5525-11e8-92e3-4775a3f5848c')
http://localhost:8080/FROST-Server/v1.0/Observations('6a7017a4-5525-11e8-92e3-8394df0a4500')
http://localhost:8080/FROST-Server/v1.0/Observations('6ac40422-5525-11e8-92e3-d725fa93c496')
http://localhost:8080/FROST-Server/v1.0/Observations('6b195242-5525-11e8-92e3-9f1179e7817c')
http://localhost:8080/FROST-Server/v1.0/Observations('6b6a2bf4-5525-11e8-92e3-6b916f5f0135')
http://localhost:8080/FROST-Server/v1.0/Observations('6bbd0752-5525-11e8-92e3-f319466b40b8')
http://localhost:8080/FROST-Server/v1.0/Observations('6c0d4bb8-5525-11

http://localhost:8080/FROST-Server/v1.0/Observations('855028c0-5525-11e8-92e3-5f5e73c207cb')
http://localhost:8080/FROST-Server/v1.0/Observations('85a44892-5525-11e8-92e3-6f48ccabc739')
http://localhost:8080/FROST-Server/v1.0/Observations('85f80f04-5525-11e8-92e3-7b02c3c10fad')
http://localhost:8080/FROST-Server/v1.0/Observations('8648f8f6-5525-11e8-92e3-5b19eebab772')
http://localhost:8080/FROST-Server/v1.0/Observations('869c150e-5525-11e8-92e3-8b53bd9bda0c')
http://localhost:8080/FROST-Server/v1.0/Observations('86ecade8-5525-11e8-92e3-dfe8dc8a0b1b')
http://localhost:8080/FROST-Server/v1.0/Observations('8741d872-5525-11e8-92e3-d701b396c3ab')
http://localhost:8080/FROST-Server/v1.0/Observations('8795c37e-5525-11e8-92e3-0b5351277210')
http://localhost:8080/FROST-Server/v1.0/Observations('87e92c80-5525-11e8-92e3-57cf7a56d02f')
http://localhost:8080/FROST-Server/v1.0/Observations('883d82da-5525-11e8-92e3-cf0cbd55e7a1')
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store 

('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 5

('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 5

http://localhost:8080/FROST-Server/v1.0/Observations('b54db5ba-5525-11e8-92e3-77ac48f8a55f')
http://localhost:8080/FROST-Server/v1.0/Observations('b5a12150-5525-11e8-92e3-43af935c0b99')
http://localhost:8080/FROST-Server/v1.0/Observations('b5f23edc-5525-11e8-92e3-a7b35cfe2bde')
http://localhost:8080/FROST-Server/v1.0/Observations('b6477f1e-5525-11e8-92e3-d3884678bc91')
http://localhost:8080/FROST-Server/v1.0/Observations('b69bc90c-5525-11e8-92e3-cba5c13ca378')
http://localhost:8080/FROST-Server/v1.0/Observations('b6f09b1c-5525-11e8-92e3-4b435cdac964')
http://localhost:8080/FROST-Server/v1.0/Observations('b74441c2-5525-11e8-92e3-cb74b5a9a985')
http://localhost:8080/FROST-Server/v1.0/Observations('b797f04c-5525-11e8-92e3-339f391d5a2c')
http://localhost:8080/FROST-Server/v1.0/Observations('b7edf03c-5525-11e8-92e3-2f267bb66c4e')
http://localhost:8080/FROST-Server/v1.0/Observations('b84176e4-5525-11e8-92e3-0b8b3095f34e')
http://localhost:8080/FROST-Server/v1.0/Observations('b891d24c-5525-11

ConnectionError: ('Connection aborted.', error(104, 'Connection reset by peer'))

In [7]:

#idDF = idDF.withColumn("FoIUID",F.format_string("%s/FeaturesOfInterest/%s",idDF.ThID,idDF.ID))
#idDF = idDF.withColumn("SensorUID",F.format_string("%s/Sensors/%s",idDF.ThID,idDF.ID))


+----------+---------------+--------------------+--------------------+
|        ID|         LatLon|               ThUID|               DsUID|
+----------+---------------+--------------------+--------------------+
|IBADENWR82|[48.441, 9.257]|edu.teco.wang/IBA...|edu.teco.wang/Dat...|
+----------+---------------+--------------------+--------------------+



[Row(ID=u'IBADENWR82', LatLon=[u'48.441', u'9.257'], ThUID=u'edu.teco.wang/IBADENWR82', DsUID=u'edu.teco.wang/Datastreams/IBADENWR82')]

In [12]:
row = idDF.collect()[0]

row

Row(ID=u'IBADENWR82', LatLon=[u'48.441', u'9.257'], ThUID=u'edu.teco.wang/IBADENWR82', DsUID=u'edu.teco.wang/Datastreams/IBADENWR82')

('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
201
201
201
('Error:', 500)
Failed to store data.


In [7]:
# Observations
dataFileDF.show(10)

+----------+--------+--------+---------+--------------+--------+-----------+----------+
|      Date|    Time|Latitude|Longitude|AirTemperature|Humidity|AirPressure|        ID|
+----------+--------+--------+---------+--------------+--------+-----------+----------+
|2016-03-02|00:00:00|  48.441|    9.257|           1.8|      80|     1014.4|IBADENWR82|
|2016-03-02|00:50:00|  48.441|    9.257|           1.8|      81|     1014.1|IBADENWR82|
|2016-03-02|01:30:00|  48.441|    9.257|           2.0|      81|     1013.1|IBADENWR82|
|2016-03-02|02:10:00|  48.441|    9.257|           2.1|      81|     1012.8|IBADENWR82|
|2016-03-02|02:55:00|  48.441|    9.257|           2.4|      81|     1011.7|IBADENWR82|
|2016-03-02|03:40:00|  48.441|    9.257|           2.6|      81|     1011.1|IBADENWR82|
|2016-03-02|04:20:00|  48.441|    9.257|           2.7|      82|     1010.7|IBADENWR82|
|2016-03-02|04:55:00|  48.441|    9.257|           2.8|      82|     1010.4|IBADENWR82|
|2016-03-02|05:30:00|  48.441|  

In [8]:
row = dataFileDF.collect()[0]
row

Row(Date=u'2016-03-02', Time=u'00:00:00', Latitude=u'48.441', Longitude=u'9.257', AirTemperature=u'1.8', Humidity=u'80', AirPressure=u'1014.4', ID=u'IBADENWR82')

In [20]:
from datetime import tzinfo, timedelta, datetime
for row in dataFileDF.collect():
    

    class TZ(tzinfo):
        def utcoffset(self, dt): return timedelta(minutes=120)
    isoTime =datetime(int(row.Date.split('-')[0]),
                 int(row.Date.split("-")[1]),
                 int(row.Date.split("-")[2]),
                 int(row.Time.split(':')[0]),
                 int(row.Time.split(':')[1]),
                 int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()
    
    
    
    
    urlObs = urlHome+"/Datastreams('edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"
    data ={
      "phenomenonTime": isoTime,
      "resultTime" : isoTime,
      "result" : row.AirTemperature
    }
    p = requests.post(urlObs, json.dumps(data))
    if (p.status_code  == 201):
        print(p.headers["location"])
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)


http://localhost:8080/FROST-Server/v1.0/Observations('c62c6f4c-54f9-11e8-a120-a38effb66126')
http://localhost:8080/FROST-Server/v1.0/Observations('c63704d4-54f9-11e8-a120-833e876e6890')
http://localhost:8080/FROST-Server/v1.0/Observations('c63c3616-54f9-11e8-a120-e3baddcdd003')
http://localhost:8080/FROST-Server/v1.0/Observations('c6414a52-54f9-11e8-a120-5fe9a4698870')
http://localhost:8080/FROST-Server/v1.0/Observations('c644b386-54f9-11e8-a120-ab95ada1cd16')
http://localhost:8080/FROST-Server/v1.0/Observations('c6487ab6-54f9-11e8-a120-2fe1351d6558')
http://localhost:8080/FROST-Server/v1.0/Observations('c64d04b4-54f9-11e8-a120-f36557bfaa37')
http://localhost:8080/FROST-Server/v1.0/Observations('c64fbd30-54f9-11e8-a120-ef16b27964a3')
http://localhost:8080/FROST-Server/v1.0/Observations('c65373d0-54f9-11e8-a120-c3cb57e03279')
http://localhost:8080/FROST-Server/v1.0/Observations('c656b36a-54f9-11e8-a120-07bbba753721')
http://localhost:8080/FROST-Server/v1.0/Observations('c65a94e4-54f9-11

In [19]:
urlObs

u"http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0/Datastreams('edu.teco.wang/IBADENWR82/Datastreams/Air-Temperature-IBADENWR82')/Observations"

In [16]:
p.headers["location"]

"http://localhost:8080/FROST-Server/v1.0/Observations('2b22bdaa-54f7-11e8-a120-4f0e65e3bb06')"

In [37]:
from datetime import tzinfo, timedelta, datetime
row = dataFileDF.first()
row.Date.split("-")[0]
class TZ(tzinfo):
    def utcoffset(self, dt): return timedelta(minutes=120)
datetime(int(row.Date.split('-')[0]),
         int(row.Date.split("-")[1]),
         int(row.Date.split("-")[2]),
         int(row.Time.split(':')[0]),
         int(row.Time.split(':')[1]),
         int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()

'2015-01-28T11:45:00+02:00'

## Tryouts

In [ ]:
row = idDF.collect()[1]
urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
data =  {
        "name": "DWD-Sensor-noSens" + row[0],
        "description": "DWD_Sensor-" + row[0],
        "@iot.id": "DWD-Sensor/noSensblblb" + row[0]
    }
p = requests.post(urlThings, json.dumps(data))
if (p.status_code  == 201):
    print(201)
else:
    print("Error:", p.status_code)
    
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [7]:

idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))#.show(3)#.agg(F.first(idDF.Longitude))

In [8]:
urlHome = 'http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0'
#urlHome = 'http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'

In [9]:

#idDF2 = idDF.select("ID","LatLon").rdd.map(DfToSensorthings)

In [27]:
row = idDF.collect()[1]
import requests
urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
sensorAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row[0]
data =  {
        "name": "DWD-Sensor-" + row[0],
        "description": "DWD_Sensor-" + row[0],
        "@iot.id": "DWD-Sensor/" + row[0]
    }
p = requests.get(urlThings+"('DWD-Sensor/noSensblbl" + row[0]+"')")
if (p.status_code  == 200):
    print(200)
else:
    print("Error:", p.status_code)
    
for chunk in p.iter_content(chunk_size=128):
    print(chunk)

200
{
  "name" : "DWD-Sensor-noSensIKNIGSBA6",
  "description" : "DWD_Sensor-IKNIGSBA6",
  "Locations@iot.navigationLink" : "http://
localhost:8080/FROST-Server/v1.0/Things('DWD-Sensor/noSensblblIKNIGSBA6')/Locations",
  "HistoricalLocations@iot.navigationLink"
 : "http://localhost:8080/FROST-Server/v1.0/Things('DWD-Sensor/noSensblblIKNIGSBA6')/HistoricalLocations",
  "Datastreams@iot.na
vigationLink" : "http://localhost:8080/FROST-Server/v1.0/Things('DWD-Sensor/noSensblblIKNIGSBA6')/Datastreams",
  "MultiDatastre
ams@iot.navigationLink" : "http://localhost:8080/FROST-Server/v1.0/Things('DWD-Sensor/noSensblblIKNIGSBA6')/MultiDatastreams",
 
 "@iot.id" : "DWD-Sensor/noSensblblIKNIGSBA6",
  "@iot.selfLink" : "http://localhost:8080/FROST-Server/v1.0/Things('DWD-Sensor/n
oSensblblIKNIGSBA6')"
}


201


<bound method Response.json of <Response [500]>>

In [27]:
row

[Row(ID=u'IKNIGSBA9', LatLon=[u'48.961', u'8.577']),
 Row(ID=u'IKNIGSBA6', LatLon=[u'48.964', u'8.616']),
 Row(ID=u'IKNIGSEG2', LatLon=[u'47.928', u'9.419']),
 Row(ID=u'IKNIGSBA8', LatLon=[u'48.965', u'8.646']),
 Row(ID=u'IKNIGSBR2', LatLon=[u'48.256', u'10.887']),
 Row(ID=u'IKNIGSBO2', LatLon=[u'52.136', u'11.770'])]

In [23]:
p = requests.post(urlThings, json.dumps(data))
if (p.status_code  == 201):
    print("Creation successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)
        

Creation successful


In [28]:
#Delete a thing

deleteID = urlThings + "('DWD-Sensors/IBREUNA2')"
print(deleteID)
p = requests.delete(urlThings)
if (p.status_code  == 201):
    print("Deletion successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

http://smartaqnet.teco.edu:8080/FROST-Server/v1.0/Things('DWD-Sensors/IBREUNA2')
('Error:', 400)
DELETE only allowed on Entities.


In [20]:
#Query a thing

getID = urlThings + "(\"teco.edu/Test-2\")"
print(getID)
p = requests.get(urlThings)
if (p.status_code  == 201):
    print("Get successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

http://smartaqnet.teco.edu:8080/FROST-Server/v1.0/Things("teco.edu/Test-2")
('Error:', 200)
{
  "value" : [ {
    "name" : "Teco test",
    "description" : "Teco test",
    "Locations@iot.navigationLink" : "http://localh
ost:8080/FROST-Server/v1.0/Things('teco.edu/Test-0')/Locations",
    "HistoricalLocations@iot.navigationLink" : "http://localhos
t:8080/FROST-Server/v1.0/Things('teco.edu/Test-0')/HistoricalLocations",
    "Datastreams@iot.navigationLink" : "http://localhos
t:8080/FROST-Server/v1.0/Things('teco.edu/Test-0')/Datastreams",
    "MultiDatastreams@iot.navigationLink" : "http://localhost:8
080/FROST-Server/v1.0/Things('teco.edu/Test-0')/MultiDatastreams",
    "@iot.id" : "teco.edu/Test-0",
    "@iot.selfLink" : "htt
p://localhost:8080/FROST-Server/v1.0/Things('teco.edu/Test-0')"
  }, {
    "name" : "Teco test",
    "description" : "Teco test"
,
    "Locations@iot.navigationLink" : "http://localhost:8080/FROST-Server/v1.0/Things('teco.edu/Test-1')/Locations",
    "Histo
rical

In [17]:
p.content

'{\n  "value" : [ {\n    "name" : "Teco test",\n    "description" : "Teco test",\n    "Locations@iot.navigationLink" : "http://localhost:8080/FROST-Server/v1.0/Things(\'teco.edu/Test-0\')/Locations",\n    "HistoricalLocations@iot.navigationLink" : "http://localhost:8080/FROST-Server/v1.0/Things(\'teco.edu/Test-0\')/HistoricalLocations",\n    "Datastreams@iot.navigationLink" : "http://localhost:8080/FROST-Server/v1.0/Things(\'teco.edu/Test-0\')/Datastreams",\n    "MultiDatastreams@iot.navigationLink" : "http://localhost:8080/FROST-Server/v1.0/Things(\'teco.edu/Test-0\')/MultiDatastreams",\n    "@iot.id" : "teco.edu/Test-0",\n    "@iot.selfLink" : "http://localhost:8080/FROST-Server/v1.0/Things(\'teco.edu/Test-0\')"\n  }, {\n    "name" : "Teco test",\n    "description" : "Teco test",\n    "Locations@iot.navigationLink" : "http://localhost:8080/FROST-Server/v1.0/Things(\'teco.edu/Test-1\')/Locations",\n    "HistoricalLocations@iot.navigationLink" : "http://localhost:8080/FROST-Server/v1.0

In [11]:
sensors = idDF.collect()
for sensor in sensors:
    print(DfToSensorthingsPost(sensor))
    #self link

Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.
Fail:Failed to store data.


In [75]:
import requests

In [16]:
dataFileDF = ss.read.parquet(inputDir)
outDir = "./outDir/outAllAll5/"
print(csv.__file__)

/gpfs/software/x86_64/anaconda/envs/jupyterhub/lib/python3.5/csv.py


In [17]:
validFileNames = [inputDir+f for f in os.listdir(inputDir) if ('.' not in f) and ("part-" in f)]
validFileNames

['./outDir/outAll4/part-00000']

In [18]:
for aFile in validFileNames:
    with open(aFile) as csvfile:
        for row in csvfile:
            k = row[1:9]
            v = row[13:-3]+'\n'
            vNew = re.sub(';','\n',v)
            outFile = open(outDir+k,"w")
            outFile.write(vNew)
            outFile.close()


In [5]:
row = csvfile.readline()

In [8]:
k = row[1:9]
v = row[13:-3].split(";")
k,v

('20150128',
 ['IKNIGSBR2,2015-01-28,06:17:00,48.256,10.887,0.5',
  'IKNIGSBR2,2015-01-28,06:48:00,48.256,10.887,0.5',
  'IKNIGSBR2,2015-01-28,07:25:00,48.256,10.887,0.4'])

In [3]:
with open(inputFile) as csvfile:
    for row in csvfile:
        k = row[1:9]
        v = row[13:-3]+'\n'
        vNew = re.sub(';','\n',v)
        outFile = open(outDir+k,"w")
        outFile.write(vNew)
        outFile.close()

NameError: name 're' is not defined

In [30]:
import re

print(vNew)

IKNIGSBA5,2015-01-29,11:14:00,48.950,8.667,3.4
IKNIGSBA8,2015-01-29,17:39:00,48.965,8.646,6.3
IKNIGSBA8,2015-01-29,18:36:00,48.965,8.646,6.1
IKNIGSBA8,2015-01-29,18:59:00,48.965,8.646,5.8
IKNIGSBA8,2015-01-29,19:40:00,48.965,8.646,5.4



In [31]:
outFile = open(outDir+k,"w")
outFile.write(vNew)
outFile.close()

In [3]:
with open(inputFile,'rb') as csvfile:
    reader = csv.Reader(csvfile)
    row = reader.__next__()
row

AttributeError: module 'csv' has no attribute 'Reader'

In [6]:
row

OrderedDict([('(20150128', '(20150129'),
             (' \'{{"id": "IKNIGSBR2"', ' \'{{"id": "IKNIGSBA5"'),
             (' "date": "2015-01-28"', ' "date": "2015-01-29"'),
             (' "time": "06:17:00"', ' "time": "11:14:00"'),
             (' "latitude": "48.256"', ' "latitude": "48.965"'),
             (' "longitude": "10.887"', ' "longitude": "8.646"'),
             (' "airtemperature": "0.5"}', ' "airtemperature": "4.2"}'),
             ('{"id": "IKNIGSBA9"', '{"id": "IKNIGSBO2"'),
             (' "time": "04:44:00"', ' "time": "17:39:00"'),
             (' "latitude": "48.961"', ' "latitude": "52.136"'),
             (' "longitude": "8.577"', ' "longitude": "11.770"'),
             (' "airtemperature": "2.2"}', ' "airtemperature": "2.6"}'),
             (' "time": "05:24:00"', ' "time": "18:36:00"'),
             (' "airtemperature": "2.3"}', ' "airtemperature": "2.0"}'),
             (' "time": "06:24:00"', ' "time": "10:06:00"'),
             (' "time": "07:44:00"', ' "tim

In [10]:
all=string.maketrans('','')
nodigs=all.translate(all, string.digits)

AttributeError: module 'string' has no attribute 'maketrans'

In [11]:
class Del:
  def __init__(self, keep=string.digits):
    self.comp = dict((ord(c),c) for c in keep)
  def __getitem__(self, k):
    return self.comp.get(k)

In [13]:
with open(inputPath) as csvfile:
    reader = csv.DictReader(csvfile)
    for i in range(10):
        row = reader.__next__()
        
        lineID = row['id']
        timeStamp = row['time'].translate(Del())
        print(lineID, timeStamp)
        

I17SAINT2 2016081912000002
I17SAINT2 2016081812000002
I17SAINT2 2016081712000002
I17SAINT2 2016081612000002
I17SAINT2 2016081512000002
I17SAINT2 2016081412000002
I17SAINT2 2016081312000002
I17SAINT2 2016081212000002
I17SAINT2 2016081112000002
I17SAINT2 2016081012000002
